In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
import agents
import sender2
import game2
from agents import find_lengths

In [2]:
NUM_OPTIONS = 10
NUM_DISTRACTORS = 9
BATCH_SIZE = 32
HIDDEN_SIZE = 44
EMBED_DIM = 44
VOCAB_SIZE = 99
MAX_LEN = 10
NUM_EPOCHS = 2
TRAINING = True
SENDER_ALL_INPUT = True
sender_entropy_coeff = 0.015 #wie bei Ossenkopf
receiver_entropy_coeff = 0.0 # wie bei Ossenkopf


In [3]:

vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)

In [4]:
#Initialisiere die Agents
sender_encoder = sender2.Sender(num_options=NUM_OPTIONS, batch_size=BATCH_SIZE)
sender_encoder_only_target = sender2.SenderOnlyTarget(batch_size=BATCH_SIZE)
sender_LSTM = sender2.Sender_LSTM(embed_dim=EMBED_DIM,
                                    num_cells=1,
                                    hidden_size=1, 
                                    max_len=MAX_LEN,
                                see_all_input=SENDER_ALL_INPUT)
receiver_encoder = agents.Receiver(num_options = NUM_OPTIONS)
receiver_LSTM = agents.Receiver_LSTM(agent=receiver_encoder, 
                                       vocab_size=VOCAB_SIZE,
                                       embed_dim=EMBED_DIM, 
                                       hidden_size=HIDDEN_SIZE)
guesser = agents.AuxiliaryNetwork(hidden_size=HIDDEN_SIZE)


In [5]:
#the losses

#loss of guessing the correct target
def loss(_sender_input, _sender_lstm_input, _message, _receiver_input, input_concepts, receiver_output, targets):
    """
    receiver_output ist was von receiver_sampling zurückgegeben wird
    LABELS PRINTEN IN OSSSENKOPF NOTEBOOK
    """
    guesses = []
    
    for i in range(len(receiver_output)):
        guesses.append(input_concepts[i][receiver_output[i]])
            
    guesses = tf.convert_to_tensor(guesses)
    targets = tf.convert_to_tensor(targets)
    acc = np.sum(guesses == targets) - np.sum(guesses != targets)
    
    return -acc

#auxiliary loss to promote empathy
def auxiliary_loss(receiver_thoughts, 
                   _message, _receiver_input, 
                   guesser_output, 
                   _labels,
                   weight=0.2):
    mae = tf.keras.losses.MeanAbsoluteError(reduction = 'none')
    loss = mae(receiver_thoughts, guesser_output)
    loss *= weight
    
    return loss

In [6]:
game = game2.Game(sender_encoder=sender_encoder,
                  sender=sender_LSTM,
                receiver=receiver_LSTM,
                main_loss=loss,
                sender_entr_coeff=sender_entropy_coeff,
                receiver_entr_coeff=receiver_entropy_coeff,
                batch_size=BATCH_SIZE,
                max_len=MAX_LEN,
                 sender_all_input=SENDER_ALL_INPUT)
print("trainable_variables:")
print(game.trainable_variables)

trainable_variables:
[]


In [7]:
optim_guesser = tf.keras.optimizers.Adam(learning_rate = 1e-3)
#optim_receiver = tf.keras.optimizers.Adam(learning_rate = 1e-2)
#optim_sender = tf.keras.optimizers.Adam(learning_rate = 1e-3)
optim_game = tf.keras.optimizers.Adam(learning_rate = 1e-5)

In [10]:
losses_network = []
losses_aux = []

for i in range(NUM_EPOCHS):
    # 1. data holen, labels?
    ####Fehlt hier noch
    data = ConceptData(voc=vocabulary, num_distractors=NUM_DISTRACTORS, batch_size=BATCH_SIZE)
    input_concepts, sender_input, targets, receiver_input = data.getInput()
    
    
    with tf.GradientTape() as tape:
        
        #print("trainable_variables:")
        #print(game.trainable_variables)
        loss, prev_hidden, last_hidden, acc, message = game(input_concepts, sender_input, targets, receiver_input)
        losses_network.append(np.mean(loss))
        game_grads = tape.gradient(loss, game.trainable_variables)
        #print("sender_train: ", game.sender.trainable_variables)
        #print("sender_encoder_grad: ", sender_encoder_gradients)
        #print("trainable variables: ", game.trainable_variables)
        #print("gradients: ", game_grads)
    
    optim_game.apply_gradients(zip(game_grads, game.trainable_variables))
    #optim_game.apply_gradients(zip(sender_lstm_gradients, game.sender.trainable_variables))
    # (grad, var) for (grad, var) in zip(game_gradients, game.trainable_variables) if grad is not None
    
    with tf.GradientTape(persistent=True) as tape:
        
        guesser_output = guesser(prev_hidden)
        aux_loss = auxiliary_loss(last_hidden, message, receiver_input, guesser_output, targets)
        losses_aux.append(aux_loss)
        aux_loss_mean = np.mean(aux_loss)
        
        aux_gradients = tape.gradient(aux_loss, guesser.trainable_variables)
        
    sender_encoder_grad_all = []
    for i in range(len(aux_gradients)):
        sender_encoder_grad_all.append(aux_gradients[i] + sender_encoder_gradients[i])
    
    optim_game.apply_gradients(zip(sender_encoder_grad_all, game.sender_encoder.trainable_variables))
    optim_guesser.apply_gradients(zip(aux_gradients, guesser.trainable_variables))
    print(f'epoch::: {i}   loss::: {np.mean(loss)}   acc::: {acc}   aux_loss::: {aux_loss_mean}')
        

ValueError: Variable <tf.Variable 'game/receiver_lstm/receiver/encoder/dense_3/kernel:0' shape=(595, 44) dtype=float32> has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [ ]:
losses_network